In [93]:
##Data Processing and Cleaning:


import pandas as pd
# Read the CSV file
df = pd.read_csv("Saudi_Arabia_hotels.csv", names=["Hotel Name", "Address", "Score", "Number of reviews", "Price", "Room type", "Star rating", "Room view", "Swimming pool", "Restaurant", "Parking", "Wheelchair accessibility"])
df = df.drop(index=0)
df





,Hotel Name,Address,Score,Number of reviews,Price,Room type,Star rating,Room view,Swimming pool,Restaurant,Parking,Wheelchair accessibility
1,Malfa Hotel Apartments,Qal'at Bishah,8.1Scored 8.1,823 reviews,"SAR 1,250",Twin Room with Private Bathroom,NaN,yes,no,no,yes,yes
2,Intour Al Khafji Hotel,Al Khafji,7.7Scored 7.7,474 reviews,"SAR 1,465",Deluxe Room,4.0,no,no,yes,yes,no
3,Braira Al Ahsa,Al Ahsa,8.9Scored 8.9,"2,750 reviews","SAR 3,071",Classic Room,4.0,yes,yes,yes,yes,no
4,الراحة بلازا للشقق المفروشة,Sharurah,7.1Scored 7.1,96 reviews,SAR 625,Twin Room,3.0,yes,no,no,yes,no
5,Boudl Majmaa,Al Majmaah,8.1Scored 8.1,700 reviews,"SAR 1,656",Standard Room With King Bed,NaN,no,yes,yes,yes,no
6,ريف الشرق للشقق الفندقية,Al Madinah,8.3Scored 8.3,309 reviews,SAR 990,Double Room,NaN,no,no,no,yes,no
7,فندق نوفا بارك,Sharurah,9.2Scored 9.2,23 reviews,SAR 650,Double Room,NaN,no,no,no,yes,yes
8,Rabigh Park Hotel,Rabigh,8.0Scored 8.0,504 reviews,"SAR 1,238",Superior Double Room,4.0,no,no,no,no,no
9,Caravan by Habitas AlUla,AlUla,9.2Scored 9.2,714 reviews,"SAR 4,872",Double Room,5.0,yes,no,yes,yes,no
10,Boudl Al Rass,Al Rass,8.7Scored 8.7,546 reviews,"SAR 1,330",Studio,NaN,no,no,no,yes,yes


In [81]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Display the count of missing values for each column
print("Missing values:\n", missing_values)


Missing values:
 Hotel Name                    0
Address                       0
Score                         3
Number of reviews             3
Price                         0
Room type                     0
Star rating                 324
Room view                     0
Swimming pool                 0
Restaurant                    0
Parking                       0
Wheelchair accessibility      0
dtype: int64


Since there are only a few missing values for Score and Number of reviews we consider filling them in with the mean.
For the Star rating Since there are quite a few missing values for this column (324), filling them in might not be appropriate. Instead we consider that it's appropriate to replace them with NaN.

In [82]:
import numpy as np
# Convert the 'Score' column to string type
df['Score'] = df['Score'].astype(str)

# Remove "Scored" from the values in the 'Score' column
df['Score'] = df['Score'].str.replace('Scored', '')

# Split the values by space and take the first score
df['Score'] = df['Score'].str.split().str[0]

# Replace "nan" with NaN
df['Score'] = df['Score'].replace('nan', np.nan)

# Convert the 'Score' column to numeric
df['Score'] = pd.to_numeric(df['Score'])

# Calculate the mean score excluding NaN values
mean_score = df['Score'].mean()

# Replace missing values in 'Score' column with the mean
df['Score'].fillna(mean_score, inplace=True)



we find diffuculty when fiilinf the score with the mean since each score is followed by (Scored), so we replace this word with "", then we convert the values into numbers after that we calculate the mean of the socre 

In [87]:
# Convert the 'Number of reviews' column to string and then replace "( reviews)" text
df['Number of reviews'] = df['Number of reviews'].astype(str).str.replace(' reviews', '')

# Convert the 'Number of reviews' column to integer type
df['Number of reviews'] = pd.to_numeric(df['Number of reviews'], errors='coerce').fillna(0).astype(int)
# Calculate the mean number of reviews excluding NaN values
mean_reviews = df['Number of reviews'].mean()

# Replace missing values in 'Number of reviews' column with the mean
df['Number of reviews'].fillna(mean_reviews, inplace=True)





In [91]:
import numpy as np

# Convert missing values in 'Star rating' column to NaN
df['Star rating'] = df['Star rating'].replace(0, np.nan)


In [92]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Display the count of missing values for each column
print("Missing values:\n", missing_values)

Missing values:
 Hotel Name                    0
Address                       0
Score                         0
Number of reviews             0
Price                         0
Room type                     0
Star rating                 324
Room view                     0
Swimming pool                 0
Restaurant                    0
Parking                       0
Wheelchair accessibility      0
dtype: int64


Missing values were addressed and the format of 'Score' and 'Number of reviews' columns standardized. Initially, text ("Scored" and "reviews") was removed from respective columns, followed by conversion to numeric type for 'Score' and integer type for 'Number of reviews'. Missing values were then filled with the mean value of each column, excluding NaN values. These steps ensured data consistency and prepared it for analysis.